# Testing Human VAE

In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import *
import numpy as np
import os

import networks
from utils import *

torch.autograd.set_detect_anomaly(True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
tdm_ckpt = 'logs/2023/04202126/tdm/models/tdm_final.pth' # input()

tdm_hyperparams = np.load(os.path.join(os.path.dirname(tdm_ckpt),'tdm_hyperparams.npz'), allow_pickle=True)
tdm_args = tdm_hyperparams['args'].item()
tdm = networks.TDM(**(tdm_hyperparams['tdm_config'].item().__dict__)).to(device)
tdm.load_state_dict(torch.load(tdm_ckpt)['model'])

vae_hyperparams = np.load(os.path.join(os.path.dirname(tdm_args.vae_ckpt),'hyperparams.npz'), allow_pickle=True)
vae_args = vae_hyperparams['args'].item()
vae = getattr(networks, vae_args.model)(**(vae_hyperparams['vae_config'].item().__dict__)).to(device)

vae.load_state_dict(torch.load(tdm_args.vae_ckpt)['model'])
vae.eval()

with np.load(tdm_args.src, allow_pickle=True) as data:
	test_data_np = data['test_data']
	test_data = [torch.Tensor(traj) for traj in test_data_np]
	test_num = len(test_data)
	print(test_num,'Testing Trajecotries')
	lens = []
	for traj in test_data:
		lens.append(traj.shape[0])
	padded_sequences = pad_sequence(test_data, batch_first=True, padding_value=1.)

39 Testing Trajecotries


In [6]:
batch_size, seq_len, dims = padded_sequences.shape
mask = torch.arange(seq_len).unsqueeze(0).repeat(batch_size,1) < lens.unsqueeze(1).repeat(1,seq_len)
x1_tdm = padded_sequences[:,:,p1_tdm_idx].to(device)
# x2_tdm = x[:,:,p2_tdm_idx].to(device)
x1_vae = padded_sequences[:,:,p1_vae_idx].to(device)
x2_vae = padded_sequences[:,:,p2_vae_idx].to(device)

x1_vae_out, _, _ = vae(x1_vae)
x2_vae_out, _, _ = vae(x2_vae)

# didn't fully understand how p(d|h_1) can be used to generate p(z_1|d) and p(z_2|d) 
# since the generated z_1 and z_2 (and x_1 and x_2) would belong to the same general distribution obtained from p(d|h_1)
#
# Is it the case that they have two separate AEs for each actor even in the HH case?
z_d1_dist, d1_samples, d1_dist = tdm(torch.nn.utils.rnn.pack_padded_sequence(x1_tdm, lens, batch_first=True, enforce_sorted=False), seq_len)
x_tdm_out = vae._output(vae._decoder(z_d1_dist.mean))


39 torch.Size([39, 1261, 965])


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from utils import *
import asyncio

fig, ax = prepare_axis()
async def update():
    global ax
    for frame_idx in range(1000):
        ax = reset_axis(ax)
        ax = visualize_skeleton(ax, test_data[frame_idx], markerfacecolor='r', linestyle='-', alpha=0.5)
        ax = visualize_skeleton(ax, x_gen[frame_idx], markerfacecolor='m', linestyle='--', alpha=0.2)

        test_data[N+frame_idx, ..., 0] = 0.7 - test_data[N+frame_idx, ..., 0]
        test_data[N+frame_idx, ..., 1] = 0.2 - test_data[N+frame_idx, ..., 1]
        x_gen[N+frame_idx, ..., 0] = 0.7 - x_gen[N+frame_idx, ..., 0]
        x_gen[N+frame_idx, ..., 1] = 0.2 - x_gen[N+frame_idx, ..., 1]

        ax = visualize_skeleton(ax, test_data[N+frame_idx], markerfacecolor='b', linestyle='-', alpha=0.5)
        ax = visualize_skeleton(ax, x_gen[N+frame_idx], markerfacecolor='g', linestyle='--', alpha=0.2)
        
        fig.canvas.draw_idle()
        fig.canvas.flush_events()
        await asyncio.sleep(0.001)
loop = asyncio.get_event_loop()
loop.create_task(update());
